# Reinforcement Learning:

In [2]:
import gym
import collections
from tensorboardX import SummaryWriter

In [3]:
ENV_NAME = "FrozenLake-v0"
GAMMA = 0.9
ALPHA = 0.2
TEST_EPISODES = 20

In [4]:
class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME) # Make environment.
        self.state = self.env.reset() # Get start state.
        self.values = collections.defaultdict(float) # Q table.

    def sample_env(self): # Gets the pair: state, action, reward, next_state.
        action = self.env.action_space.sample()
        old_state = self.state
        new_state, reward, is_done, _ = self.env.step(action)
        self.state = self.env.reset() if is_done else new_state
        return (old_state, action, reward, new_state)

    def best_value_and_action(self, state): # Chooses best action among many possible.
        best_value, best_action = None, None
        for action in range(self.env.action_space.n): # For all action.
            action_value = self.values[(state, action)] # Get the Q value for state-action.
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_value, best_action

    def value_update(self, s, a, r, next_s): # Bellman update with Blending.
        best_v, _ = self.best_value_and_action(next_s)
        new_val = r + GAMMA * best_v
        old_val = self.values[(s, a)]
        self.values[(s, a)] = old_val * (1-ALPHA) + new_val * ALPHA

    def play_episode(self, env): # Run simulation.
        total_reward = 0.0
        state = env.reset()
        while True:
            _, action = self.best_value_and_action(state) # Get best action for current state.
            new_state, reward, is_done, _ = env.step(action) # Perform action.
            total_reward += reward # Store reward.
            if is_done:
                break
            state = new_state
        return total_reward

In [5]:
if __name__ == "__main__":
    test_env = gym.make(ENV_NAME)
    agent = Agent()
    writer = SummaryWriter(comment="-q-learning")

    iter_no = 0
    best_reward = 0.0
    while True:
        iter_no += 1
        s, a, r, next_s = agent.sample_env()
        agent.value_update(s, a, r, next_s)

        reward = 0.0
        for _ in range(TEST_EPISODES):
            reward += agent.play_episode(test_env)
        reward /= TEST_EPISODES
        writer.add_scalar("reward", reward, iter_no)
        if reward > best_reward:
            print("Best reward updated %.3f -> %.3f" % (best_reward, reward))
            best_reward = reward
        if reward > 0.80:
            print("Solved in %d iterations!" % iter_no)
            break
    writer.close()

Best reward updated 0.000 -> 0.250
Best reward updated 0.250 -> 0.350
Best reward updated 0.350 -> 0.400
Best reward updated 0.400 -> 0.450
Best reward updated 0.450 -> 0.500
Best reward updated 0.500 -> 0.550
Best reward updated 0.550 -> 0.600
Best reward updated 0.600 -> 0.650
Best reward updated 0.650 -> 0.800
Best reward updated 0.800 -> 0.900
Solved in 2836 iterations!
